In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_pacf

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#dl libraraies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("Vinicius Marques Matos - co2-ppm-mauna-loa-19651980.csv")

FileNotFoundError: ignored

In [ ]:
df.columns = ['year-month','ppm']

In [ ]:
df = df[:-1]

In [ ]:
df['year'] = [line[:-3] for line in df['year-month']]
df['ppm'] = df['ppm'].astype(int)
df['year'] = df['year'].astype(int)
df['month'] = [line[5:] for line in df['year-month']]
df['month'] = df['month'].astype(int)


In [ ]:
df.shape

In [ ]:
def shift_test(lags):
    df_new = df.copy()# Create a copy of the DataFrame
    for i in range(1,lags+1):# Shift the "y" values by the specified number of lags
        df_new[f"ppm{i}"] = df_new["ppm"].shift(i)
        df_new.dropna(inplace=True)# Remove rows with missing values (NaN)
        df_new[f"ppm{i}"] = df_new[f"ppm{i}"].astype(int)

    return df_new # Convert the data to numpy arrays and return them

In [ ]:
shift_test(5).tail()

In [ ]:
size = int((df.shape[0]) * 0.9)# Calculate the size for the training se
df_new = shift_test(5)
df_new = df_new[df_new.columns.drop('year-month')]
x_train, y_train, x_test, y_test = df_new[df_new.columns.drop('ppm')][:size],df_new['ppm'][:size], df_new[df_new.columns.drop('ppm')][size:],df_new['ppm'][size:]

In [ ]:
num_classes = 1

model = Sequential()
model.add(Dense(25, input_shape=(7,)))
model.add(Dense(8))
model.add(Dense(4))
model.add(Dense(num_classes))

model.summary()


In [ ]:
df_new

In [ ]:
model.compile(loss='mse',
              optimizer="RMSprop",
              metrics=['mae'])

batch_size = 50
num_classes = 3
epochs = 20

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1
                    # ,                    validation_data=(x_test, y_test)
                    )

In [ ]:
# Plotando o historico do processo de treinamento

plt.figure(figsize=(20, 5))
plt.plot(history.history['loss'], color='blue')
# plt.plot(history.history['val_loss'], color='red')
plt.title('Model loss', fontsize=20)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Treinamento', 'Validação'], loc='upper right', fontsize=14)
plt.show()

In [ ]:
plt.plot(pd.to_datetime(df['year-month']),pd.to_numeric(df['ppm']))

In [ ]:
year='1965'
plt.plot(df[df['year']==year]['month'],pd.to_numeric(df[df['year']==year]['ppm']))

In [ ]:
df['ppm'] = df['ppm'].astype('float64')

In [ ]:
a = [line for line in df['ppm']]


In [ ]:
a = a[:-1]

In [ ]:
a.insert(0, '')

In [ ]:
a

In [ ]:
df['last'] = a

In [ ]:
a = df.groupby('year').mean()

In [ ]:
df['last_year'] = [a.loc[str(int(year)-1)]['ppm']  if int(year) > int(df['year'].min()) else '' for year in df['year']]

In [ ]:
df

In [ ]:
 df[df['year']=='1965']